For the approximate solution, the script constructs a complete graph from the given set of points, finds a Minimum Spanning Tree (MST) using Kruskal's algorithm, performs a minimum weight perfect matching for vertices with odd degrees, and finally, finds an Eulerian tour to approximate a solution to TSP. This approach is based on heuristic methods to quickly find a near-optimal solution.

The approximate solution is faster but may not always yield the optimal solution, while the exact solution guarantees optimality but may be computationally expensive for larger instances.

In [13]:
import math
import heapq
import networkx as nx
import numpy as np

# Function to solve the Traveling Salesman Problem (TSP)
def tsp(data, method='heuristic'):
    # If the number of data points is small, switch to exact method
    # if len(data) <= 3:
    #     method = 'exact'

    # Choose the appropriate method based on input
    if method == 'heuristic':
        return tsp_heuristic(data)
    elif method == 'exact':
        return tsp_exact(data)
    else:
        raise ValueError("Method must be either 'heuristic' or 'exact'")

# Heuristic method for solving TSP
def tsp_heuristic(data):
    # Build graph from given distance matrix
    G = build_graph_from_matrix(data)

    # Find Minimum Spanning Tree (MST) using Kruskal's algorithm
    MSTree = minimum_spanning_tree(G)

    # Find vertices with odd degrees in MST
    odd_vertexes = find_odd_vertexes(MSTree, G)

    # Perform minimum weight perfect matching for odd degree vertices
    augmented_MST = minimum_weight_matching(G, odd_vertexes)

    # Find an Eulerian tour in the augmented MST
    eulerian_tour = find_eulerian_tour(augmented_MST, G)

    # Create a Hamiltonian circuit from the Eulerian tour
    return create_hamiltonian_circuit(eulerian_tour, G)

# Function to build a complete graph from given distance matrix

# Function to calculate the Euclidean distance between two points
def get_length(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

def build_graph_from_matrix(data):
    n = len(data)
    graph = {i: {} for i in range(n)}
    for i in range(n):
        for j in range(i + 1, n):
            graph[i][j] = graph[j][i] = data[i][j]
    return graph

# Class for implementing Union-Find data structure
class UnionFind:
    def __init__(self):
        self.parent = {}

    def find(self, i):
        if i not in self.parent:
            self.parent[i] = i
        while i != self.parent[i]:
            i = self.parent[i] = self.parent[self.parent[i]]
        return i

    def union(self, i, j):
        self.parent[self.find(i)] = self.find(j)

# Function to find Minimum Spanning Tree (MST) using Kruskal's algorithm
def minimum_spanning_tree(G):
    edges = [(cost, u, v) for u, neighbors in G.items() for v, cost in neighbors.items() if u < v]
    heapq.heapify(edges)
    tree, subtrees = [], UnionFind()
    while edges and len(tree) < len(G) - 1:
        cost, u, v = heapq.heappop(edges)
        if subtrees.find(u) != subtrees.find(v):
            tree.append((u, v))
            subtrees.union(u, v)
    return tree

# Function to find vertices with odd degrees in MST
def find_odd_vertexes(MST, G):
    degree = {v: 0 for v in G}
    for u, v in MST:
        degree[u] += 1
        degree[v] += 1
    return [v for v, d in degree.items() if d % 2 == 1]

# Function to perform minimum weight perfect matching for odd degree vertices
def minimum_weight_matching(G, odd_vert):
    subgraph = nx.Graph()
    for v in odd_vert:
        for u in odd_vert:
            if u != v:
                subgraph.add_edge(v, u, weight=-G[v][u])  # Negate weight for max weight matching
    matching = nx.max_weight_matching(subgraph, maxcardinality=True)
    return [(u, v) for u, v in matching]

# Function to find an Eulerian tour in the augmented MST
def find_eulerian_tour(MST, G):
    neighbors = {v: [] for v in G}
    for u, v in MST:
        neighbors[u].append(v)
        neighbors[v].append(u)

    start_vertex = max(neighbors, key=lambda v: len(neighbors[v]))
    stack, path = [start_vertex], []
    while stack:
        u = stack[-1]
        if neighbors[u]:
            v = neighbors[u].pop()
            neighbors[v].remove(u)
            stack.append(v)
        else:
            path.append(stack.pop())
    return path

# Function to create a Hamiltonian circuit from an Eulerian tour
def create_hamiltonian_circuit(eulerian_tour, G):
    path, visited = [], set()
    for v in eulerian_tour:
        if v not in visited:
            path.append(v)
            visited.add(v)
    path.append(path[0])  # Complete the circuit
    return path

def extract_tour(x, n):
    tour = [0]  # Start from the first node
    current = 0
    for _ in range(1, n):
        for j in range(n):
            if j != current and x[(current, j)].varValue == 1:  # Fix indexing issue
                tour.append(j)
                current = j
                break
    return tour


# Function to calculate the total distance of the tour
def calculate_total_distance(tour, data):
    total_distance = sum(get_length(data[tour[i-1]][0], data[tour[i-1]][1], data[tour[i]][0], data[tour[i]][1]) for i in range(1, len(tour)))
    total_distance += get_length(data[tour[-1]][0], data[tour[-1]][1], data[tour[0]][0], data[tour[0]][1])
    return total_distance

# Define your dataset as a distance matrix
data_matrix = np.array([[0, 3, 4, 2],
                        [3, 0, 1, 4],
                        [4, 1, 0, 5],
                        [2, 4, 5, 0]])  # Add more points as needed

# Solve TSP (method param exact or heuristic, default is heuristic)
length, path = tsp(data_matrix, "exact")

print(f"Path: {path}")  # Path will be a list of integers representing the order of cities visited
print(f"Total length: {length}")

# Approximate the upper bound using the approximation factor (3/2)
upper_bound = length * (3/2)

# You can add your own code here to calculate the lower bound

print(f"Approximate Upper Bound optimal path distance: {upper_bound}")


Path: [0, 3, 2, 1]
Total length: 11.498473502456159
Approximate Upper Bound optimal path distance: 17.24771025368424


Integration of path deduced by christofides algorithm into Genetic Algorithm by setting it as the initial population.

In [10]:
import random
import copy

class GeneticAlgorithmTSP:
    def __init__(self, graph, population_size=10, mutation_rate=0.01, generations=100):
        self.graph = graph
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.generations = generations

    def greedy_path(self, s, path, visited):
        visited[s] = True
        adj = self.graph[s]
        minimum = float('inf')
        child = -1
        for i in range(len(adj)):
            if not visited[i] and adj[i] < minimum and adj[i] != 0:
                child = i
                minimum = adj[i]
        if child != -1:
            path.append(child)
            self.greedy_path(child, path, visited)

    def shortest_path(self, source):
        size = len(self.graph)
        path = [source]
        visited = [False] * size
        self.greedy_path(source, path, visited)
        print(path)
        return path

    def initial_population(self):
        population =[]
        num_cities = len(self.graph)
        for _ in range(self.population_size):
            # route = list(range(num_cities))
            # random.shuffle(route)
            population.append(path)
        # print(population)
        return population

    def distance(self, city1, city2):
        return self.graph[city1][city2]

    def fitness(self, route):
        total_distance = 0
        for i in range(len(route) - 1):
            total_distance += self.distance(route[i], route[i + 1])
        total_distance += self.distance(route[-1], route[0])
        return 1 / total_distance

    def selection(self, population):
        ranked_routes = [(self.fitness(route), route) for route in population]
        ranked_routes.sort(reverse=True)
        return ranked_routes[0][1]

    def breed(self, parent1, parent2):
        start = random.randint(0, len(parent1) - 1)
        end = random.randint(0, len(parent1) - 1)
        if start > end:
            start, end = end, start

        child = [-1] * len(parent1)
        for i in range(start, end + 1):
            child[i] = parent1[i]

        j = 0
        for i in range(len(parent2)):
            if j == start:
                j = end + 1
            if parent2[i] not in child:
                child[j] = parent2[i]
                j += 1

        return child

    def mutate(self, route):
        for i in range(len(route)):
            if random.random() < self.mutation_rate:
                j = random.randint(0, len(route) - 1)
                route[i], route[j] = route[j], route[i]

    def evolve(self):
        population = self.initial_population()
        for i in range(self.generations):
            best_route = self.selection(population)
            print(f"Generation {i + 1}, Best route: {best_route}")
            new_population = [best_route]
            for _ in range(1, self.population_size):
                parent1_index = random.randint(0, len(population) - 1)
                parent2_index = random.randint(0, len(population) - 1)
                child = self.breed(population[parent1_index], population[parent2_index])
                self.mutate(child)
                new_population.append(child)
            population = copy.deepcopy(new_population)

if __name__ == "__main__":
    random.seed(42)  # for reproducibility

    graph = data_matrix

    tsp_ga = GeneticAlgorithmTSP(graph)
    tsp_ga.evolve()


Generation 1, Best route: [0, 3, 2, 1]
Generation 2, Best route: [0, 3, 2, 1]
Generation 3, Best route: [0, 3, 2, 1]
Generation 4, Best route: [0, 3, 2, 1]
Generation 5, Best route: [0, 3, 2, 1]
Generation 6, Best route: [0, 3, 2, 1]
Generation 7, Best route: [0, 3, 2, 1]
Generation 8, Best route: [2, 3, 0, 1]
Generation 9, Best route: [2, 3, 0, 1]
Generation 10, Best route: [3, 0, 2, 1]
Generation 11, Best route: [3, 0, 2, 1]
Generation 12, Best route: [3, 0, 2, 1]
Generation 13, Best route: [3, 0, 2, 1]
Generation 14, Best route: [3, 0, 2, 1]
Generation 15, Best route: [3, 0, 2, 1]
Generation 16, Best route: [3, 0, 2, 1]
Generation 17, Best route: [3, 0, 2, 1]
Generation 18, Best route: [3, 0, 2, 1]
Generation 19, Best route: [3, 1, 2, 0]
Generation 20, Best route: [3, 1, 2, 0]
Generation 21, Best route: [3, 1, 2, 0]
Generation 22, Best route: [3, 1, 2, 0]
Generation 23, Best route: [3, 1, 2, 0]
Generation 24, Best route: [3, 1, 2, 0]
Generation 25, Best route: [3, 1, 2, 0]
Generatio